In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import itertools

### Conditionals

For imputation of MAR data (all labels wrt .docx code book)

#### TB

- If TB2 = 5, then, TB2_4_TEXT = -1, TB3 = 1, TB4 = 1
- If TB3 = 1, then TB4 = 1
- If TB5 = 1, then TB6 = -1, TB7 = 1, TB8 = 1
- If TB7 = 1, then TB8 = 1
- If TB9 = 1, then TB10 = -1, TB11 = 1, TB12 = 1
- If TB11 = 1, then TB12 = 1

Combine TB2 and TB2_4_TEXT into one (take TB2 only)

#### AL

- If AL1 = 2, then AL1_4_TEXT = -1, AL2_1_TEXT = -1, AL3_1_TEXT = -1, AL4 = 1, AL5 = 1, AL6A = 1, AL6B = 1
    - If AL1 = 3, then AL1_4_TEXT = -2
- If AL2 = 2 and/or AL3 = 2, then AL2_1_TEXT = -2 and/or AL3_1_TEXT = -2
- If AL5 = 1, then AL6A = 1, AL6B = 1

Combine AL1 and AL1_4_TEXT, AL2 and AL2_1_TEXT, AL3 and AL3_1_TEXT, AL6A and AL6B (take AL1, AL2, AL3, AL6A only)

#### ID and ND

- If ID1 = 1, then ID2 = -1, ID3-12 = 1, ID15-20 = -1
- If ID3 = 1, then ID4-12 = 1, ID15-20 = -1 (wrong labels in C1W1, changed it back in terms of .docx labels beforehand)
- If ID17 = 1, then ID18-20 = -1
- If ND1 = 1, then ND2 = -1

#### DA

Multiple choice variable: ['DA5','DA6','DA7','DA7a-d','DA8','DA8a-d'] --> Create new variable for each choice made by participants

- If DA7_1 = 0, then DA7ax = 0
- If DA7_15 = 0, then DA7bx = 0
- If DA7_16 = 0, then DA7cx = 0
- If DA7_17 = 0, then DA7dx = 0
- If DA7_18 = 1, then DA7ax = 0, DA7bx = 0, DA7cx = 0, DA7dx = 0

<br>

- If DA8_17 = 0, then DA8ax = 0
- If DA8_18 = 0, then DA8bx = 0
- If DA8_19 = 0, then DA8cx = 0
- If DA8_20 = 0, then DA8dx = 0
- If DA8_21 = 1, then DA8ax = 0, DA8bx = 0, DA8cx = 0, DA8dx = 0

#### OD, CJ, and DM

- If OD1 = 1, then OD2 = 1
- If OD6 = 2, then OD7-11 = 1
- If OD8 = 1, then OD9 = 1
- If OD10 = 1, then OD11 = 1
- If CJ3 = 1, then CJ4-7 = -1
- If DM12 != 1, then DM13 = -1

In [2]:
def impute_MARs(vars, df):

    new_vars = []
    for v in vars:
        col = df[v]
        if v == 'TB2':
            for idx, i in enumerate(col):
                if i == 5:
                    df.at[idx, 'TB2_4_TEXT'] = -1
                    df.at[idx, 'TB3'] = 1
                    df.at[idx, 'TB4'] = 1
        if v == 'TB3':
            for idx, i in enumerate(col):
                if i == 1:
                    df.at[idx, 'TB4'] = 1
        if v == 'TB5':
            for idx, i in enumerate(col):
                if i == 1:
                    df.at[idx, 'TB6'] = -1
                    df.at[idx, 'TB7'] = 1
                    df.at[idx, 'TB8'] = 1
        if v == 'TB7':
            for idx, i in enumerate(col):
                if i == 1:
                    df.at[idx, 'TB8'] = 1
        if v == 'TB9':
            for idx, i in enumerate(col):
                if i == 1:
                    df.at[idx, 'TB10'] = -1
                    df.at[idx, 'TB11'] = 1
                    df.at[idx, 'TB12'] = 1
        if v == 'TB11':
            for idx, i in enumerate(col):
                if i == 1:
                    df.at[idx, 'TB12'] = 1
        if v == 'AL1':
            for idx, i in enumerate(col):
                if i == 2:
                    df.at[idx, 'AL1_4_TEXT'] = -1
                    df.at[idx, 'AL2_1_TEXT'] = -1
                    df.at[idx, 'AL3_1_TEXT'] = -1
                    df.at[idx, 'AL4'] = 1
                    df.at[idx, 'AL5'] = 1
                    df.at[idx, 'AL6A'] = 1
                    df.at[idx, 'AL6B'] = 1
                elif i == 3:
                    df.at[idx, 'AL1_4_TEXT'] = -2
        if v == 'AL2':
            for idx, i in enumerate(col):
                if i == 2:
                    df.at[idx, 'AL2_1_TEXT'] = -2
        if v == 'AL3':
            for idx, i in enumerate(col):
                if i == 2:
                    df.at[idx, 'AL3_1_TEXT'] = -2
        if v == 'AL5':
            for idx, i in enumerate(col):
                if i == 1:
                    df.at[idx, 'AL6A'] = 1
                    df.at[idx, 'AL6B'] = 1
        if v == 'AL6A':
            for idx, i in enumerate(col):
                if pd.isnull(df.loc[idx, v]) and not pd.isnull(df.loc[idx, 'AL6B']):
                    df.at[idx, v] = df.at[idx, 'AL6B']
        if v == 'ID1':
            for idx, i in enumerate(col):
                if i == 1:
                    df.at[idx, 'ID2'] = -1
                    for j in range(3,13):
                        df.at[idx, f'ID{j}'] = 1
                    for j in range(15,21):
                        df.at[idx, f'ID{j}'] = -1
        if v == 'ID3':
            for idx, i in enumerate(col):
                if i == 1:
                    for j in range(4,13):
                        df.at[idx, f'ID{j}'] = 1
                    for j in range(15,21):
                        df.at[idx, f'ID{j}'] = -1
        if v == 'ID17':
            for idx, i in enumerate(col):
                if i == 1:
                    for j in range(18,21):
                        df.at[idx, f'ID{j}'] = -1
        if v == 'ND1':
            for idx, i in enumerate(col):
                if i == 1:
                    df.at[idx, 'ND2'] = -1
        if v in ['DA5','DA6','DA7','DA7a','DA7b','DA7c','DA7d','DA8','DA8a','DA8b','DA8c','DA8d']:
            col_nonan = col.dropna()
            choices = sorted(set(list(itertools.chain.from_iterable([entry.split(',') for entry in col_nonan]))), key=int)  # all choices made
            names = [f'{v}_{c}' for c in choices]
            new_vars.extend(names)
            for j, c in enumerate(choices):
                newcol = []
                for idx, i in enumerate(col):
                    if not pd.isnull(df.loc[idx, v]):
                        newcol.append(1) if c in i.split(',') else newcol.append(0)
                    else:   newcol.append(np.nan)
                df.insert(df.columns.get_loc(v)+j+1,names[j],newcol)
        if v == 'OD1':
            for idx, i in enumerate(col):
                if i == 1:
                    df.at[idx, 'OD2'] = 1
        if v == 'OD6':
            for idx, i in enumerate(col):
                if i == 2:
                    for j in range(7,12):
                        df.at[idx, f'OD{j}'] = 1
        if v == 'OD8':
            for idx, i in enumerate(col):
                if i == 1:
                    df.at[idx, 'OD9'] = 1
        if v == 'OD10':
            for idx, i in enumerate(col):
                if i == 1:
                    df.at[idx, 'OD11'] = 1
        if v == 'CJ3':
            for idx, i in enumerate(col):
                if i == 1:
                    for j in range(4,8):
                        df.at[idx, f'CJ{j}'] = -1
        if v == 'DM12':
            for idx, i in enumerate(col):
                if i != 1:
                    df.at[idx, 'DM13'] = -1

    for v in new_vars:
        col = df[v]
        if v == 'DA7_1':
            for idx, i in enumerate(col):
                if i == 0:
                    for vv in [name for name in list(df.columns) if name.startswith('DA7a_') and 'TEXT' not in name]:
                        df.at[idx, vv] = 0
        if v == 'DA7_15':
            for idx, i in enumerate(col):
                if i == 0:
                    for vv in [name for name in list(df.columns) if name.startswith('DA7b_') and 'TEXT' not in name]:
                        df.at[idx, vv] = 0
        if v == 'DA7_16':
            for idx, i in enumerate(col):
                if i == 0:
                    for vv in [name for name in list(df.columns) if name.startswith('DA7c_') and 'TEXT' not in name]:
                        df.at[idx, vv] = 0
        if v == 'DA7_17':
            for idx, i in enumerate(col):
                if i == 0:
                    for vv in [name for name in list(df.columns) if name.startswith('DA7d_') and 'TEXT' not in name]:
                        df.at[idx, vv] = 0
        if v == 'DA7_18':
            for idx, i in enumerate(col):
                if i == 1:
                    for vv in [name for name in list(df.columns) if name.startswith(('DA7a_','DA7b_','DA7c_','DA7d_')) and 'TEXT' not in name]:
                        df.at[idx, vv] = 0
        if v == 'DA8_17':
            for idx, i in enumerate(col):
                if i == 0:
                    for vv in [name for name in list(df.columns) if name.startswith('DA8a_') and 'TEXT' not in name]:
                        df.at[idx, vv] = 0
        if v == 'DA8_18':
            for idx, i in enumerate(col):
                if i == 0:
                    for vv in [name for name in list(df.columns) if name.startswith('DA8b_') and 'TEXT' not in name]:
                        df.at[idx, vv] = 0
        if v == 'DA8_19':
            for idx, i in enumerate(col):
                if i == 0:
                    for vv in [name for name in list(df.columns) if name.startswith('DA8c_') and 'TEXT' not in name]:
                        df.at[idx, vv] = 0
        if v == 'DA8_20':
            for idx, i in enumerate(col):
                if i == 0:
                    for vv in [name for name in list(df.columns) if name.startswith('DA8d_') and 'TEXT' not in name]:
                        df.at[idx, vv] = 0
        if v == 'DA8_21':
            for idx, i in enumerate(col):
                if i == 1:
                    for vv in [name for name in list(df.columns) if name.startswith(('DA8a_','DA8b_','DA8c_','DA8d_')) and 'TEXT' not in name]:
                        df.at[idx, vv] = 0


    vars_mixed = ['TB2_4_TEXT','TB6','TB10','AL1_4_TEXT','AL2_1_TEXT','AL3_1_TEXT','ID2','ND2']

    for v in vars_mixed:
        col = df[v]
        if v[-4:] == 'TEXT':  # e.g., modify TB2 column instead of TB2_4_TEXT
            v = v.split('_')[0]
        for idx, i in enumerate(col):
            if 0 <= i <= 14:    df.at[idx, v] = 0  # children
            elif 15 <= i <= 24: df.at[idx, v] = 1  # youth
            elif 25 <= i <= 64: df.at[idx, v] = 2  # adult
            elif i >= 65:       df.at[idx, v] = 3  # senior
            elif i == -1:       df.at[idx, v] = 4  # never
            elif i == -2:       df.at[idx, v] = 5  # don't know

    return df

### Cohort 1

In [2]:
datapath = 'data/original/'
csv_file_nonet = datapath + 'C1W1 and C1W2 NonNetwork Data.csv'
csv_file_net = datapath + 'C1W1 C1W2 With Network No NickInit_orig.csv'

In [3]:
df_nonet = pd.read_csv(csv_file_nonet)
df_net = pd.read_csv(csv_file_net)
# print(df_net.to_string)

In [4]:
fields_nonet = list(df_nonet.columns)
fields_net = list(df_net.columns)

print(len(fields_nonet))
print(len(fields_net))

342
537


Below blocks verifies that non-network data is subset of network data

In [ ]:
for var in fields_nonet:
    if var not in fields_net:
        print(f'variable {var} is exclusive to non-network data')
    # else:
    #     print(f'variable {var} is also included in network data')

In [ ]:
for i in [1, 10, 11, 12, 13, 100]:
    field = fields_nonet[i]
    print(f"Field '{field}' in column {fields_nonet.index(field)+1} of non-network data is in column {fields_net.index(field)+1} of network data")

Field 'C1W1_PID2' in column 2 of non-network data is in column 2 of network data
Field 'C1W1_DM1' in column 11 of non-network data is in column 206 of network data
Field 'C1W1_TB1' in column 12 of non-network data is in column 207 of network data
Field 'C1W1_TB2' in column 13 of non-network data is in column 208 of network data
Field 'C1W1_TB2_4_TEXT' in column 14 of non-network data is in column 209 of network data
Field 'C1W1_DA8a' in column 101 of non-network data is in column 296 of network data


Divide data into separate waves. Note that for Aim 1A, network data for wave 2 is unavailable.

The organization of columns in (with network) csv datafile is: (wave1, non-network, SC vars) --> (wave1, network) --> (wave1, non-network, the rest) --> (wave2, non-network)

Categorize variables in txt files. Note that variable 'ND' is ambiguous: it stands for both 'network: drug use' (network) and 'non-injection drugs' (non-network).

In [5]:
C1W1nonet_idx, C1W1nonet_vars = [], []
C1W2nonet_idx, C1W2nonet_vars = [], []
C1W1net_idx, C1W1net_vars = [], []

f1 = open(datapath + "C1W1_nonnetwork.txt", "w")  # wave 1, non-network
f2 = open(datapath + "C1W2_nonnetwork.txt", "w")  # wave 2, non-network
f3 = open(datapath + "C1W1_network.txt", "w")  # wave 1, network
for field in fields_net:

    affi = field[0:4]
    varname = field[5:]
    if field in fields_nonet:
        if affi == 'C1W1':
            f1.write(varname + '\n')
            if varname == 'TB1O':
                C1W1nonet_vars.append('TB10')
            elif varname == 'US':
                C1W1nonet_vars.append('ND1')
            elif varname == 'AC34A':
                C1W1nonet_vars.append('AC3A')
            else:
                C1W1nonet_vars.append(varname)
            C1W1nonet_idx.append(fields_net.index(field))
        elif affi == 'C1W2':
            f2.write(varname + '\n')
            C1W2nonet_vars.append(varname)
            C1W2nonet_idx.append(fields_net.index(field))
    elif field not in fields_nonet and affi == 'C1W1':
        f3.write(varname + '\n')
        if varname == 'ND1':
            C1W1net_vars.append('NSD1')
        else:
            C1W1net_vars.append(varname)
        C1W1net_idx.append(fields_net.index(field))
f1.close()
f2.close()
f3.close()

C1W1nonet_df = df_net.iloc[:,C1W1nonet_idx]
C1W2nonet_df = df_net.iloc[:,C1W2nonet_idx]
C1W1net_df = df_net.iloc[:,C1W1net_idx]

# Remove prefixes from column names
C1W1nonet_df.columns = C1W1nonet_vars
C1W2nonet_df.columns = C1W2nonet_vars
C1W1net_df.columns = C1W1net_vars

Start with wave 1

Non-network (only include those available in csv)
- SC1-7: Community participants live in
- SC8-14: Social support
- DM1-4: Age, gender, sexual orientation demographics
- TB1-12: Tobacco use
- AL1-7: Alcohol use
- ID1-30: Injection drug use
- ND1-16: Non-injection drug use
- DA1-10: Drug accessibility and use patterns
- OD1-11: Drug overdose
- TX1-7: Substance use treatment
- AC1-14: Adverse childhood experiences
- CJ1-8: Criminal justice involvement
- DM5-26: Long form demographics

Network
- NS: Network - ppl they can confide in
- ND: Network - ppl they use drugs with

Ensure consistency of data type e.g., no strings in numerical variables before imputing MARs

In [10]:
C1W1nonet_df = impute_MARs(C1W1nonet_vars, C1W1nonet_df)

In [6]:
# empty cell to NaN
# C1W1nonet_df.replace(r'^\s+$', np.nan, regex=True)
# C1W2nonet_df.replace(r'^\s+$', np.nan, regex=True)
# C1W1net_df.replace(r'^\s+$', np.nan, regex=True)

C1W1nonet_df.to_csv(datapath + 'C1W1_nonnetwork.csv', index=False)
C1W2nonet_df.to_csv(datapath + 'C1W2_nonnetwork.csv', index=False)
C1W1net_df.to_csv(datapath + 'C1W1_network.csv', index=False)

# C1W1nonet_df.to_csv(datapath + 'pre-imputed/C1W1_nonnetwork_preimputed.csv', index=False)
# C1W2nonet_df.to_csv(datapath + 'pre-imputed/C1W2_nonnetwork_preimputed.csv', index=False)
# C1W1net_df.to_csv(datapath + 'pre-imputed/C1W1_network_preimputed.csv', index=False)

### Cohort 2

In [5]:
datapath = 'data/original/'
csv_file = datapath + 'C2W1 C2W2 9_30_2022 With Network No NickInit_orig.csv'
df = pd.read_csv(csv_file)
# print(df_net.to_string)

In [6]:
fields = list(df.columns)
print(len(fields))

719


Organization is: C2W2 -> C2W1

Again, wave 2 doesn't have AC features; but for Cohort 2, both waves have network features

In [14]:
C2W1nonet_idx, C2W1nonet_vars = [], []
C2W2nonet_idx, C2W2nonet_vars = [], []
C2W1net_idx, C2W1net_vars = [], []
C2W2net_idx, C2W2net_vars = [], []

f1 = open(datapath + "C2W1_nonnetwork.txt", "w")  # wave 1, non-network
f2 = open(datapath + "C2W2_nonnetwork.txt", "w")  # wave 2, non-network
f3 = open(datapath + "C2W1_network.txt", "w")  # wave 1, network
f4 = open(datapath + "C2W2_network.txt", "w")  # wave 2, network

for field in fields:

    affi = field[0:4]
    varname = field[5:]
    if affi == 'C2W1':
        if not varname.startswith('NS') and not varname.startswith('NDX'):  # wave 1, non-network
            f1.write(varname + '\n')
            if varname == 'TB1O':
                C2W1nonet_vars.append('TB10')
            elif varname == 'US':
                C2W1nonet_vars.append('ND1')
            elif varname == 'AC34A':
                C2W1nonet_vars.append('AC3A')
            else:
                C2W1nonet_vars.append(varname)
            C2W1nonet_idx.append(fields.index(field))
        else:  # wave 1, network
            f3.write(varname + '\n')
            C2W1net_vars.append(varname)
            C2W1net_idx.append(fields.index(field))
    elif affi == 'C2W2':
        if not varname.startswith('NS') and not varname.startswith('NDX'):  # wave 2, non-network
            f2.write(varname + '\n')
            if varname == 'TB1O':
                C2W2nonet_vars.append('TB10')
            elif varname == 'US':
                C2W2nonet_vars.append('ND1')
            elif varname == 'AC34A':
                C2W2nonet_vars.append('AC3A')
            else:
                C2W2nonet_vars.append(varname)
            C2W2nonet_idx.append(fields.index(field))
        else:  # wave 2, network
            f4.write(varname + '\n')
            C2W2net_vars.append(varname)
            C2W2net_idx.append(fields.index(field))
    
f1.close()
f2.close()
f3.close()
f4.close()

C2W1nonet_df = df.iloc[:,C2W1nonet_idx]
C2W2nonet_df = df.iloc[:,C2W2nonet_idx]
C2W1net_df = df.iloc[:,C2W1net_idx]
C2W2net_df = df.iloc[:,C2W2net_idx]

# Remove prefixes from column names
C2W1nonet_df.columns = C2W1nonet_vars
C2W2nonet_df.columns = C2W2nonet_vars
C2W1net_df.columns = C2W1net_vars
C2W2net_df.columns = C2W2net_vars

In [12]:
C2W1nonet_df = impute_MARs(C2W1nonet_vars, C2W1nonet_df)

/tmp/ipykernel_391/1875228192.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(df.columns.get_loc(v)+j+1,names[j],newcol)
/tmp/ipykernel_391/1875228192.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(df.columns.get_loc(v)+j+1,names[j],newcol)
/tmp/ipykernel_391/1875228192.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To ge

In [14]:
C2W2nonet_df = impute_MARs(C2W2nonet_vars, C2W2nonet_df)

In [15]:
C2W1nonet_df.to_csv(datapath + 'C2W1_nonnetwork.csv', index=False)
C2W2nonet_df.to_csv(datapath + 'C2W2_nonnetwork.csv', index=False)
C2W1net_df.to_csv(datapath + 'C2W1_network.csv', index=False)
C2W2net_df.to_csv(datapath + 'C2W2_network.csv', index=False)

# C2W1nonet_df.to_csv(datapath + 'pre-imputed/C2W1_nonnetwork_preimputed.csv', index=False)
# C2W2nonet_df.to_csv(datapath + 'pre-imputed/C2W2_nonnetwork_preimputed.csv', index=False)
# C2W1net_df.to_csv(datapath + 'pre-imputed/C2W1_network_preimputed.csv', index=False)
# C2W2net_df.to_csv(datapath + 'pre-imputed/C2W2_network_preimputed.csv', index=False)

We see that variables from both waves are consistent (have one-to-one mapping, although the orderings are different), for both network and non-network data (if discard 5 new variables in C2W2 and 25 AC variables in C2W1)

In [40]:
C2W1nonet_vars = [var for var in C2W1nonet_vars if not var.startswith('AC')]
C2W2nonet_vars = [var for var in C2W2nonet_vars if var not in ['ND17','ND18','ND19','ND20','ND21']]
sorted(C2W1nonet_vars) == sorted(C2W2nonet_vars)

True

In [16]:
a = [1,2,3,4, 2,1]
[i for i,d in enumerate(a) if d==2]

[1, 4]

New data for Cohort 2 (11/14): $80\rightarrow 105$ samples

In [7]:
datapath = 'data/original/'
csv_file_1114 = datapath + 'C2W1 C2W2 11_14_2022.csv'
df_1114 = pd.read_csv(csv_file_1114)

fields = list(df_1114.columns)
print(f'Old: {len(df)}; new: {len(df_1114)}. Both have the same number of columns? {len(list(df)) == len(list(df_1114))}, which is {len(fields)}')

Old: 80; new: 105. Both have the same number of columns? True, which is 719


In [8]:
# copy the partipants that previously have been cleaned to new df for consistency and time saving

pids = list(df['C2W2_PID'])
for p in pids:
    df_1114.loc[df_1114['C2W2_PID'] == p] = df.loc[df['C2W2_PID'] == p].values.flatten().tolist()

df_1114.to_csv(csv_file_1114, index=False)

In [9]:
C2W1nonet_idx, C2W1nonet_vars = [], []
C2W2nonet_idx, C2W2nonet_vars = [], []
C2W1net_idx, C2W1net_vars = [], []
C2W2net_idx, C2W2net_vars = [], []


for field in fields:

    affi = field[0:4]
    varname = field[5:]
    if affi == 'C2W1':
        if not varname.startswith('NS') and not varname.startswith('NDX'):  # wave 1, non-network
            if varname == 'TB1O':
                C2W1nonet_vars.append('TB10')
            elif varname == 'US':
                C2W1nonet_vars.append('ND1')
            elif varname == 'AC34A':
                C2W1nonet_vars.append('AC3A')
            else:
                C2W1nonet_vars.append(varname)
            C2W1nonet_idx.append(fields.index(field))
        else:  # wave 1, network
            C2W1net_vars.append(varname)
            C2W1net_idx.append(fields.index(field))
    elif affi == 'C2W2':
        if not varname.startswith('NS') and not varname.startswith('NDX'):  # wave 2, non-network
            if varname == 'TB1O':
                C2W2nonet_vars.append('TB10')
            elif varname == 'US':
                C2W2nonet_vars.append('ND1')
            elif varname == 'AC34A':
                C2W2nonet_vars.append('AC3A')
            else:
                C2W2nonet_vars.append(varname)
            C2W2nonet_idx.append(fields.index(field))
        else:  # wave 2, network
            C2W2net_vars.append(varname)
            C2W2net_idx.append(fields.index(field))


C2W1nonet_df = df_1114.iloc[:,C2W1nonet_idx]
C2W2nonet_df = df_1114.iloc[:,C2W2nonet_idx]
C2W1net_df = df_1114.iloc[:,C2W1net_idx]
C2W2net_df = df_1114.iloc[:,C2W2net_idx]

# Remove prefixes from column names
C2W1nonet_df.columns = C2W1nonet_vars
C2W2nonet_df.columns = C2W2nonet_vars
C2W1net_df.columns = C2W1net_vars
C2W2net_df.columns = C2W2net_vars

In [10]:
C2W1nonet_df.to_csv(datapath + 'pre-imputed/221114/C2W1_nonnetwork_preimputed.csv', index=False)
C2W2nonet_df.to_csv(datapath + 'pre-imputed/221114/C2W2_nonnetwork_preimputed.csv', index=False)
C2W1net_df.to_csv(datapath + 'pre-imputed/221114/C2W1_network_preimputed.csv', index=False)
C2W2net_df.to_csv(datapath + 'pre-imputed/221114/C2W2_network_preimputed.csv', index=False)

In [11]:
len(C2W1nonet_vars)

174

In [12]:
from helper import impute_MARs

df_w1 = impute_MARs(C2W1nonet_vars, C2W1nonet_df)
df_w2 = impute_MARs(C2W2nonet_vars, C2W2nonet_df)
vars = ['TB4','TB8','TB12','AL5'] + [f'ID{i}' for i in range(4,13)] + [f'ND{i}' for i in range(1,13) if i !=2]
df_w1 = df_w1[vars]
df_w2 = df_w2[vars]

for i, var in enumerate(vars):
    df_w1.insert(df_w1.columns.get_loc(var)+1, f'W2_{vars[i]}', list(df_w2[vars[i]]))

df_w1.to_csv(datapath + 'pre-imputed/221114/C2_nonnetwork_pred.csv', index=False)